In [ ]:
import pandas as pd
import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt
import seaborn as sns
from pihatsu import cmds

sns.set()
%matplotlib inline

cmds = ['RPM', 'SPEED', 'THROTTLE_POS', 'ENGINE_LOAD', 'COOLANT_TEMP', 'INTAKE_TEMP', 'INTAKE_PRESSURE', 'TIMING_ADVANCE']

In [ ]:
def get_avg(d):
    return d[d!=0].mean()

def get_stops(d):
    return "".join(map(str, d["SPEED"].values))

def read_data(csv_file):
    """
    Read data, do a bunch of cleaning, returns DataFrame with floats
    """
    d = pd.read_csv(csv_file, sep='\t', index_col=0, parse_dates=True)
    d = d[cmds]

    if "FUEL_STATUS" in d.keys():
        d["FUEL_STATUS"] = d["FUEL_STATUS"].astype("category").cat.codes  # to ints, avoid breaking astype(float)

    d.apply(lambda x: x.replace(regex=True, inplace=True, to_replace=r'[^0-9.]', value=""))
    d.replace('', '0', inplace=True)
    d = d[~d.index.isnull()][:-1].astype(float)
    
    return d

In [ ]:
d = read_data('data/2019-01-26T16:38:16.tsv')

In [ ]:
colors = cm.Dark2(np.linspace(0,1, len(cmds)))
figwidth = max(5, int(len(d.index) / 40))
figheight = len(cmds)*2
fig = plt.figure(figsize=(figwidth, figheight))

for i in range(len(cmds)):
    cmd, color = cmds[i], colors[i]
    ax = plt.subplot(len(cmds), 1, i+1, frameon=False, title=cmd)
    
    ax.set_xticks([])
    ax.set_yticks([])
    
    xmax, ymax = d[cmd][5:-5].idxmax(), d[cmd][5:-5].max()
    ax.annotate("max: {0:.0f}".format(ymax), 
                xy=(xmax, ymax), 
                xytext=(xmax, min(ymax*1.25, ymax+10)), 
                arrowprops=dict(facecolor=color))
    
    '''
    xmin, ymin = d[cmd][10:-5].idxmin(), d[cmd][10:-5].min()
    ax.annotate("min: {0:.1f}".format(ymin), 
                xy=(xmin, ymin), 
                xytext=(xmin, max(ymin*.75, ymin-10)), 
                arrowprops=dict(facecolor=color))
    '''
    
    xstart, ystart = d[cmd].index[0], d[cmd][0]
    ax.annotate("start: {0:.0f}".format(ystart), 
                xy=(xstart, ystart), 
                xytext=(xstart, ystart), 
                arrowprops=dict(facecolor=color))

    xend, yend = d[cmd].index[-1], d[cmd][-1]
    ax.annotate("end: {0:.0f}".format(yend), 
                xy=(xend, yend),
                xytext=(xend, yend),
                arrowprops=dict(facecolor=color))

    avg = get_avg(d[cmd])
    ax.plot(d[cmd], color=color, alpha=.75)
    ax.axhline(y=avg, color=color, linestyle='--', linewidth=.5, alpha=.5)

    avg_str = "{0:.0f}".format(avg)
    ax.annotate("avg: {}".format(avg_str), xy=(xend, avg), xytext=(xend, avg))

plt.tight_layout()
plt.savefig('sparklines.pdf')